In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import time
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
from tqdm.notebook import tqdm
from sklearn.cluster import MeanShift, KMeans
from IPython.core.debugger import set_trace
from matplotlib import pyplot, dates


In [ ]:
english_names = pd.read_csv('data/articles.csv', header=None)
english_names.columns = ['Title']
english_names['Title'] = english_names['Title'].apply(lambda x : x.replace(' ', '_'))
english_names

In [ ]:
def get_qid_from_title(title, language):
    response = requests.get(f'https://{language}.wikipedia.org/w/api.php?'
                            f'action=query&prop=pageprops&titles={title}&redirects&format=json')
    try:
        r = [item for item in response.json()['query']['pages'].values()][0]
        qid = r['pageprops']['wikibase_item']
    except KeyError:
        print(f'Article {title} has no Wikidata ID')
        return None
    return qid

### Retrieve wikidata entries for each given article

In [ ]:
qids = pd.DataFrame([get_qid_from_title(title, 'en') for title in tqdm(english_names.Title.values)])
qids.head()

### English pageview data

In [ ]:
def day_year_to_date(year, days):
    return datetime.datetime(year, 1, 1) + datetime.timedelta(days - 1)

def get_wikishark_id(article_name, lang):
    response = requests.get(f'https://www.wikishark.com/autocomplete.php?q={article_name}')
    
    r = response.json()
    
    target = None
    for candidate in r:
        if '(' + lang + ')' in candidate['name'] and article_name.replace('_',' ').lower() in candidate['name'].lower():
            return candidate['id']
    return target

def get_daily_pageviews(titles, language, start, end):
    data = []
    for title in tqdm(titles):
        
        # authors requested we wait 1 second at least in between requests
        time.sleep(1)
        
        wikishark_id = get_wikishark_id(title, language)
        if wikishark_id is None:
            print(f'Could not find data for {title}')
            continue
        # wait to avoid overloading servers
        time.sleep(1)
        
        response = requests.get(f'https://www.wikishark.com/getdata/daily.php?value={wikishark_id}?view=2&scale=0&normalized=0&loglog=0&log=0&zerofix=0')

        daily_data = response.json()
        
        # add data with timestamps
        start_date = datetime.datetime.strptime(start, '%d/%m/%Y')
        end_date   = datetime.datetime.strptime(end, '%d/%m/%Y')
        current_date = datetime.datetime.now()
        
        # wikishark returns daily page views for every day since 2007-12-31 (independent of given parameters)
        # we need to index it according to the time period we are interested in
        start_index = (len(daily_data) - 1) - (current_date - start_date).days
        end_index = (len(daily_data) - 1) - (current_date - end_date).days
        timestamps = {}
        for i, d in enumerate(daily_data[start_index:end_index]):
            ts = start_date + datetime.timedelta(days=i)
            timestamps[ts] = int(d)
    
        data.append({**{'Article': title}, **timestamps})
    
    return pd.DataFrame(data)

In [ ]:
import os

PICKLED = './privacy.pkl'

if os.path.isfile(PICKLED):
    privacy_en = pd.read_pickle(PICKLED)
else:
    privacy_en = get_daily_pageviews(list(english_names.Title), 'en', '01/01/2011','01/01/2016')
    # use article name as index
    privacy_en.index = privacy_en.Article
    privacy_en = privacy_en.drop(['Article'], axis=1)
    privacy_en.to_pickle(PICKLED)

### Aggregate into monthly data

In [ ]:
privacy_en.columns = pd.to_datetime(privacy_en.columns)

# take monthly cumulative
monthly_en = privacy_en.resample('M', axis=1).sum()
monthly_en.head()

# Exploratory data analysis 

In [ ]:
#Melt dataframe to use dates as entry values rather than columns
monthly_en_melt = pd.melt(monthly_en, value_name='views', var_name='date', ignore_index=False)
len(monthly_en_melt.index.unique())

# Quick preprocessing of articles

Since we have a lot of articles, we want to first remove articles that are of no interest to us, before we can visualize the total dataset correctly

### Get all articles that have a very low amount of views no matter what

In [ ]:
low_views = [article for article in monthly_en_melt.index.unique() if (monthly_en_melt.loc[article].views < 100).all()]
print(low_views)
monthly_en_melt = monthly_en_melt.drop(low_views)

### Get all articles that may be outliers

In the same way the original paper found outlier like the hamas article, we want to remove from our dataset sudden changes in pageviews that are not part of our signal.
Since the views an article gets monthly doesn't follow a gaussian distribution, we use IQRs to determine if an article is an outlier rather than just using the 3 standard deviations rule. If the views in a month are higher than the 75th percentile + 3 times the interquartile range, we consider it for manual inspection.

In [ ]:
def is_outlier(article, factor=2):
    q25, q75 = np.percentile(monthly_en_melt.loc[article].views, 25), np.percentile(monthly_en_melt.loc[article].views, 75)
    iqr = q75 - q25
    cut_off = iqr * factor
    lower, upper = q25 - cut_off, q75 + cut_off
    return ((monthly_en_melt.loc[article].views < lower) | (monthly_en_melt.loc[article].views > upper)).any()

outliers = [article for article in monthly_en_melt.index.unique() if is_outlier(article)]
print(len(outliers),outliers)

## Manual inspection

In order to verify that our potential outliers should be removed, we manually check each of these articles, the news related to it, to find out if there is an obvious reason for the sudden change in views, and if this reason is related to our signal : privacy enhancing technologies.

The articles we suspect as outliers are :
- I2P : Spike happens around the time Silkroad (drug network) moves from tor to I2p
- IMule : High variance all around, bump in 2012 probably related to the bump in use of i2p in 2012
- Operation onymous : Spikes around the time of operation, an international law enforcement operation targeting darknet markets and other hidden services operating on the Tor network.

**TODO : ADD YOURS**

## Outlier removal

In [ ]:
manual_outliers = ['4chan','I2P', 'IMule', 'Operation_Onymous']
monthly_en_melt= monthly_en_melt.drop(manual_outliers)

# Finally some Data visualization

### Plotting per article views (total)

In [ ]:
plt.figure(figsize=(20,5))
most_views = monthly_en_melt.reset_index().groupby('Article')['views'].sum().sort_values(ascending=False)
head = list(most_views.head(5).index) #a surprise tool that will help us later
most_views.plot.bar()
plt.title("Total pageviews per article")
plt.ylabel("Total pageviews")
plt.show()

Now that we have removed all those outlier articles, we have a look at what is the real distribution of pageviews for articles that should represent privacy enhancing techniques. We remark that there is still a high gap in the pageviews, for example, the Tor article represents a good percentage of the mass of the distribution.

It might look like this tail is quite long, but this is actually because there is a decent amount of articles that were created during the time period of our analysis, and thus don't accumulate as much total views.

## Plotting total and head 5 articles pageviews

In [ ]:
reveal = datetime.datetime(2013, 6, 5) #Around june 
fig, ax = plt.subplots(sharey=True, figsize=(10,5))
monthly_en_melt.reset_index().groupby('date').sum().reset_index().plot.scatter(x='date',y='views',ax=ax,label='All articles')
monthly_en_melt.loc[head].reset_index().groupby('date').sum().reset_index().plot.scatter(x='date',y='views',ax=ax,c='r',label='Top 5 articles')
plt.axvline(reveal,c='r')
plt.title("Total pageviews for privacy-enhancing technology articles")
plt.show()

There seems to be a lot of variance even before the reveal date, with a peak at the reveal date rather than an effect due to the reveal. We see that the top 5 articles have a very high effect on the trend.

Something interesting to note is that in the long term, there seems to be a reversing trend : the pageviews tend to go down until 2016.

## Plotting the pageviews of articles with the most weight

Since the first 5 articles are responsible for a good portion of the trend of the data, we investigate further on these.

In [ ]:
fig, axs = plt.subplots(ncols=5, figsize=(30,5), sharey=True)
for i,ax in enumerate(axs.flatten()):
    monthly_en_melt.loc[head[i]].plot.scatter(x='date',y='views',ax=ax)
    ax.axvline(reveal,c='r')
    ax.set_title(head[i])
plt.suptitle('Article pageviews for top 5 articles (with respect to total views)')
plt.show()

For the DuckDuckGo article, we are able to see quite an important direct impact on the pageviews, starting from the reveal, however, long term it doesn't seem to affect the trend.

We also see an impact on the pageviews for Tor, but since the variance is very high for that article, it is hard to have a definitive answer.

For the PGP article, there might be a little bit of an impact, but since there was already a trend of increase it is hard to say.

For the SOCKS and PeerGuardian article we can't identify any visual trend in views due to the reveal

# Segmented Regression Analysis

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
df = monthly_en_melt.groupby('date').sum()

#times before and after the reveal
before = df.loc[:reveal].reset_index()
after  = df.loc[reveal:].reset_index()

#set dates to num so regplot is able to produce a result
before.date = mdates.date2num(before.date)
after.date  = mdates.date2num(after.date)

#regplot plots both scatter, and regression fit
sns.regplot(x='date',y='views',ax=ax,data=before,label='before')
sns.regplot(x='date',y='views',ax=ax,data=after,label='after')

#reveal vertical line

plt.axvline(reveal,c='r')
plt.legend()
plt.title("Total pageviews for privacy-enhancing technology articles")

#turn numbers back to dates on the axis
loc = mdates.AutoDateLocator()
ax.xaxis.set_major_locator(loc)
ax.xaxis.set_major_formatter(mdates.AutoDateFormatter(loc))

We remark a sudden impact in the following few months, but the long term trend doesn't seem to be very much impacted by this reveal. It is interesting to compare this to the global wikipedia trend to see how it compares

# Statsmodel Regression Fit

# Comparison with the global wikipedia pageviews

# Comparison with the same articles in different languages : French

# Other Experiments :

We have tried various other ways for dealing with data with a high difference in pageviews, i.e. to not fit our whole hypothesis on the 3 articles that have the most views. The two following experiments didn't lead to significant conclusions but we leave them here for completeness.

## First Experiment : Standardizing per article

In this part, we standardize the views of each articles, thus giving the same weights to all articless.

In [ ]:
def standardize(x):
    mean = monthly_en_melt.reset_index().groupby('Article').mean()['views'].loc[x['Article']]
    std  = monthly_en_melt.reset_index().groupby('Article').std()['views'].loc[x['Article']]
    std = std if std != 0 else 1
    return (x['views'] - mean)/std

monthly_en_melt = monthly_en_melt.reset_index()
monthly_en_melt['standardized'] = monthly_en_melt.apply(standardize,axis=1)
monthly_en_melt = monthly_en_melt.set_index('Article')
monthly_en_melt.head()

### Standardized vs total views

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,4))

axs[0].axvline(reveal,c='r')
axs[1].axvline(reveal,c='r')

axs[0].set_title('Standardized')
axs[1].set_title('Views')

monthly_en_melt.groupby('date').sum().reset_index().plot.scatter(x='date', y='standardized', ax=axs[0], rot=90)
monthly_en_melt.groupby('date').sum().reset_index().plot.scatter(x='date', y='views',ax=axs[1],rot=90)

plt.tight_layout()

Given that we have a high amount of articles, even with our quick preprocessing, we have too many articles that have a low amount of views, which makes them inherently have more variance, so it is unfair to give as much weight to these articles as to the other ones.

## Second Experiment : Stratification

Instead of analyzing a single group of articles that have highly different views, we can select groups of articles that have similar viewcounts, and analyze them together.
Given that our distribution of total pageviews per articles can be considered as heavy-tailed, we probably don't want equal width or equal frequency discretization. Instead we opt for clustering to divide our articles into multiple groups, by creating clusters based on the total amount of views.

In [ ]:
sorted_views = monthly_en_melt.reset_index().groupby('Article').sum().sort_values('views',ascending=False)
vals = np.log(sorted_views['views'].values.reshape((-1,1))) #Use log scales values because they are too far away

### Selecting number of clusters

Since we don't know exactly how many groups we want to define, we will use the silhouette score and the elbow method to help us.

In [ ]:
def plot_sse(features_X, start=2, end=11):
    sse = []
    for k in range(start, end):
        # Assign the labels to the clusters
        kmeans = KMeans(n_clusters=k, random_state=10).fit(features_X)
        sse.append({"k": k, "sse": kmeans.inertia_})

    sse = pd.DataFrame(sse)
    # Plot the data
    plt.plot(sse.k, sse.sse)
    plt.xlabel("K")
    plt.ylabel("Sum of Squared Errors")
    
plot_sse(vals)

In [ ]:
from sklearn.metrics import silhouette_score
silhouettes = []

# Try multiple k
for k in range(2, 11):
    # Cluster the data and assigne the labels
    labels = KMeans(n_clusters=k, random_state=10).fit_predict(vals)
    # Get the Silhouette score
    score = silhouette_score(vals, labels)
    silhouettes.append({"k": k, "score": score})
    
# Convert to dataframe
silhouettes = pd.DataFrame(silhouettes)

# Plot the data
plt.plot(silhouettes.k, silhouettes.score)
plt.xlabel("K")
plt.ylabel("Silhouette score")

Given that there is now particular elbow, we take the value with the best silhouette score, K = 5.

In [ ]:
NUM_CLUSTERS = 5

clustering = KMeans(n_clusters=NUM_CLUSTERS).fit(vals)

plt.figure(figsize=(20,5))
plt.scatter(x=range(len(sorted_views)),y=sorted_views['views'], c=clustering.labels_)
plt.gca().set_yscale('log')
plt.xlabel('Article')
plt.ylabel('Views')
plt.gca().set_xticks(range(len(sorted_views)))
plt.gca().set_xticklabels(sorted_views.index, rotation=90)

sorted_views['group'] = clustering.labels_
monthly_en_melt['group'] = sorted_views['group']

## Plotting per group

In [ ]:
reveal = datetime.date(2013,6,5)
fig, axs = plt.subplots(nrows=NUM_CLUSTERS, figsize=(10,20))

for g,ax in zip(range(len(monthly_en_melt.group.unique())),axs.flatten()):
    #Group by date, select the group
    selected = monthly_en_melt[monthly_en_melt['group'] == g]
    df = selected.groupby('date').sum()

    before = df.loc[:reveal].reset_index()
    after  = df.loc[reveal:].reset_index()
    before.date = mdates.date2num(before.date)
    after.date  = mdates.date2num(after.date)
    
    sns.regplot(x='date',y='views',ax=ax,data=before)
    sns.regplot(x='date',y='views',ax=ax,data=after)
    
    loc = mdates.AutoDateLocator()
    ax.xaxis.set_major_locator(loc)
    ax.xaxis.set_major_formatter(mdates.AutoDateFormatter(loc))
    
    sample = selected.reset_index().groupby('Article').sum().sort_values('views', ascending=False).head(3).index
    sample_str = ','.join(sample)
    ax.set_title(f'Group {g} ({sample_str})')
plt.tight_layout()

Unfortunately, we are not very comfortable on interpreting these results, since they show many different trends, so we would have to go through each individual article and verify what causes the trend, so we also stopped with this analysis. Another issue we thought of is that the clusters we would find on other languages of wikipedia would be different so it would be hard to compare a general trend given that we find no general trend here.